pip install selenium

In [57]:
from selenium import webdriver
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.by import By


# Inciando a seção e abrindo a tabela de Cataguases
driver = webdriver.Chrome()

try:

    driver.get("https://www.emater.mg.gov.br/portal.do?flagweb=novaintranet_pgn_pmater_educ_lista_uregi")

    select_element = driver.find_element(By.ID,'uregi')
    select_object = Select(select_element)

    select_object.select_by_index(5)

    search_button = driver.find_element(By.ID, "ok")
    search_button.click()


    #Pegando o arquivo e as infos de cada linha

    n = 2

    lista_info = []

    for i in range(1,6):
        lista_info.append(driver.find_element(By.XPATH, f"/html/body/section/div/div[1]/table/tbody/tr[2]/td[{i}]").text)
        print(driver.find_element(By.XPATH, f"/html/body/section/div/div[1]/table/tbody/tr[2]/td[{i}]").text)


    print(lista_info)

    
    print ("E a data?")

    #município = driver.find_element(By.XPATH, "/html/body/section/div/div[1]/table/tbody/tr[n]/td[1]").text
    #regional_ensino = driver.find_element(By.XPATH, "/html/body/section/div/div[1]/table/tbody/tr[n]/td[2]").text
    #meta = driver.find_element(By.XPATH, "/html/body/section/div/div[1]/table/tbody/tr[n]/td[3]").text
    #cliente = driver.find_element(By.XPATH, "/html/body/section/div/div[1]/table/tbody/tr[n]/td[4]").text
    
    data = driver.find_element(By.XPATH, f"/html/body/section/div/div[1]/table/tbody/tr[1]/td[5]")
    print(data)
    
    data1 = driver.find_element(By.XPATH, f"/html/body/section/div/div[1]/table/tbody/tr[2]/td[7]/p[2]")
    print(data.get_attribute("number"))
    print(data.get_attribute("date"))
    print(data.get_attribute("value"))
    print(data.get_attribute("text"))
    print(data.get_attribute("name"))
    print(data.get_property("number"))
    print(data.get_property("date"))
    print(data.get_property("text"))
    print(data.get_property("name"))
    print(data.id)
    print(data.text)
    
    
finally:
    driver.quit()


Rodeiro
Ubá
Projeto 2 - Meta 2.3 - ATER: Produção, Organização e Mercado
José Marcio de Mendonça

['Rodeiro', 'Ubá', 'Projeto 2 - Meta 2.3 - ATER: Produção, Organização e Mercado', 'José Marcio de Mendonça', '']
E a data?
<selenium.webdriver.remote.webelement.WebElement (session="9fdbe52546fe5887b24fd77311a3f0c1", element="4a2af696-8336-4443-a640-2cb5497f6d73")>
None
None
None
None
None
None
None
None
None
4a2af696-8336-4443-a640-2cb5497f6d73



In [3]:
from selenium import webdriver
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys


# Inciando a seção e abrindo a tabela de Cataguases
driver = webdriver.Chrome()

try:

    driver.get("https://www.emater.mg.gov.br/portal.do?flagweb=novaintranet_pgn_pmater_educ_lista_uregi")

    select_element = driver.find_element(By.ID,'uregi')
    select_object = Select(select_element)

    select_object.select_by_index(5) # Posição na lista de seleção

    search_button = driver.find_element(By.ID, "ok")
    search_button.click()


    #Pegando o arquivo e as infos de cada linha
    

    """
    n = 2

    lista_info = []

    for i in range(1,6):
        lista_info.append(driver.find_element(By.XPATH, f"/html/body/section/div/div[1]/table/tbody/tr[2]/td[{i}]").text)
        print(driver.find_element(By.XPATH, f"/html/body/section/div/div[1]/table/tbody/tr[2]/td[{i}]").text)


    print(lista_info)
    """
   
    # Com essa porcaria consegui pegar todos os dados de uma vez
    elements = driver.find_elements(By.TAG_NAME, 'tr')
    i = 0
    for e in elements:
        print(e.text)
        #driver.execute_script("window.scrollTo(0, 300)")
        #driver.execute_script("rola = rola + 1000;")
        #driver.execute_script("window.scrollTo({top: rola,behavior: 'smooth'});")
        #objeto = driver.find_element(By.CLASS_NAME, "textoCentral")
        #objeto.sendKeys(Keys.DOWN)
        i += e.rect['height']
        driver.execute_script(f"window.scrollTo(0,{i})")

finally:
    pass
    #driver.quit()
  

Município Regional Ensino Meta Cliente Data Documentos Enviado SEI
Rodeiro Ubá Projeto 2 - Meta 2.3 - ATER: Produção, Organização e Mercado José Marcio de Mendonça 23/11/2021
09/02/2022
Rodeiro Ubá Projeto 2 - Meta 2.3 - ATER: Produção, Organização e Mercado Edvar Nogueira Junior 24/11/2021
Rodeiro Ubá Projeto 2 - Meta 2.3 - ATER: Produção, Organização e Mercado Evaldo José Nogueira 02/12/2021
Itamarati de Minas Leopoldina Projeto 2 - Meta 2.3 - ATER: Produção, Organização e Mercado Marcio Luiz Porto Pereira 16/11/2021
Itamarati de Minas Leopoldina Projeto 2 - Meta 2.3 - ATER: Produção, Organização e Mercado Bráz Pereira Dias 16/11/2021
Itamarati de Minas Leopoldina Projeto 2 - Meta 2.3 - ATER: Produção, Organização e Mercado Genésio Ferreira do Nascimento 16/11/2021
Itamarati de Minas Leopoldina Projeto 2 - Meta 2.3 - ATER: Produção, Organização e Mercado Jaime de Oliveira Dutra 17/11/2021
Itamarati de Minas Leopoldina Projeto 2 - Meta 2.3 - ATER: Produção, Organização e Mercado Nelci

In [37]:
from selenium import webdriver

driver = webdriver.Chrome()

# Navigate to url
driver.get("https://www.example.com")

    # Returns height, width, x and y coordinates referenced element
res = driver.find_element(By.CSS_SELECTOR, "h1").rect

print(res)

{'height': 43, 'width': 600, 'x': 217, 'y': 133.4375}


In [9]:
from selenium import webdriver
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.by import By


# Inciando a seção e abrindo a tabela de Cataguases
driver = webdriver.Chrome()
driver.maximize_window()
original_window = driver.current_window_handle


driver.get("https://www.emater.mg.gov.br/portal.do?flagweb=novaintranet_pgn_pmater_educ_lista_uregi")

select_element = driver.find_element(By.ID,'uregi')
select_object = Select(select_element)

select_object.select_by_index(5)

search_button = driver.find_element(By.ID, "ok")
search_button.click()


#Pegando o arquivo e as infos de cada linha e salvando em um dicionário

list_info = []

for i in range(1,6):
    list_info.append(driver.find_element(By.XPATH, f"/html/body/section/div/div[1]/table/tbody/tr[2]/td[{i}]").text)
    #print(driver.find_element(By.XPATH, f"/html/body/section/div/div[1]/table/tbody/tr[2]/td[{i}]").text)
print(list_info)

#Clicando na imagem do arquivo
driver.find_element(By.XPATH, f"/html/body/section/div/div[1]/table/tbody/tr[2]/td[6]").click()

driver.switch_to.window(driver.window_handles[1])

driver.find_element(By.LINK_TEXT, "Download").click()

#Close the tab or window
driver.close()

#Switch back to the old tab or window
driver.switch_to.window(original_window)
    
    
finally:
    pass
    #driver.quit()


['Rodeiro', 'Ubá', 'Projeto 2 - Meta 2.3 - ATER: Produção, Organização e Mercado', 'José Marcio de Mendonça', '23/11/2021']
